# Web scrapping for Data Scientist job in CO (9 points total)



In this exercise we will do web scrapping for **Data Scientist job in CO**


Here is the link to the search query

https://www.indeed.com/jobs?q=data+scientist&l=CO

As you can see at the bottom of the page there are links to several pages related to this search.
If you click on second page, search url changes to

https://www.indeed.com/jobs?q=data+scientist&l=CO&start=10

If you click on 3rd then url changes to

https://www.indeed.com/jobs?q=data+scientist&l=CO&start=20

Hence, to go to more pages we can format the search string(**change start=??** part) for **requests.get in a loop**


# Q1(5 =  4(non indicator columns) + 1(indicator columns) points) Please complete the following task

- Scrape 10 pages (**last page(10 th) url will be https://www.indeed.com/jobs?q=data+scientist&l=CO&start=90**)and build a pandas DataFrame containing following information
    + **job title, name of the company, location, summary of job description**
    + **Indicator columns(with value True/False) about keywords Python, SQL, AWS, RESTFUL, Machine learning, Deep Learning, Text Mining, NLP, SAS, Tableau, Sagemaker, TensorFlow, Spark**

Note:
- Make sure that you do a case insensitive search for keywords when filing(Tue/False) in the indicator columns
- You need to go to the webpage of detail job posting for keywords search. The main job posting only contains summary of the job description.  Build detail job posting webpage url from web scrapping main search results.
- If you run into difficulties which you are not able to overcome, skip this question and import the datafram from the provided the pickle file instead.
- If you find this entire homework too difficult at your current level of expertise, please feel free to complete the AlternateHwk5 instead.

In [1]:
from bs4 import BeautifulSoup as bsoup
import requests
import csv
import pandas as pd

In [5]:
site = "https://www.indeed.com/jobs?q=data+scientist&l=CO&start="
Jobs = {}
skills = ['python', 'sql', 'aws', 'restful', 'machine learning', 'deep learning', 'text mining', 
        'nlp', 'sas', 'tableau', 'sagemaker', 'tensorflow', 'spark']

for i in range( 0, 100, 10):
    sites = site + str(i)
    pages = requests.get(sites)
    soup = bsoup(pages.text, 'html.parser')
    sliders = soup.find_all('div', 'slider_container')
    
    
    for i in range(len(sliders)):
        for name in sliders[i].find_all('span', title=True):
            jb_id = sliders[i].find('a')['id'].split('_')[-1]
            Jobs[jb_id]={'job_title': name['title'], 
                         'company': sliders[i].find('span', 'companyName').text, 
                         'location': sliders[i].find('div', 'companyLocation').text, 
                         'job_description': sliders[i].find('div', 'job-snippet').text.strip("\n")}
            
            id_ = sliders[i].find('a')['data-jk']
            idurl = 'https://www.indeed.com/viewjob?viewtype=embedded&jk=' + id_
            idpage = requests.get(idurl)
            soup_id = bsoup(idpage.content, 'html.parser')
            
            try:
                Jobs[jb_id]['temp'] = soup_id.select_one('#jobDescriptionText').get_text(strip=True, separator='')
            except AttributeError:
                Jobs[jb_id]['temp'] = Jobs[jb_id]['job_description']
            
            for skill in skills:
                if skill in Jobs[jb_id]['temp'].lower():
                    Jobs[jb_id][skill] = True
                else:
                    Jobs[jb_id][skill] = False

In [6]:
df = pd.DataFrame.from_dict(Jobs, orient = 'index')
df = df.drop(columns=['temp'])

In [7]:
jobs_df = df.rename(columns = {'python':'Python', 'sql':'SQL', 'aws':'AWS', 'restful':'RESTFUL', 'machine learning':'Machine_Learning', 'deep learning':'Deep_Learning', 'text mining':'Text_Mining', 
        'nlp':'NLP', 'sas':'SAS', 'tableau':'Tableau', 'sagemaker':'Sagemaker', 'tensorflow':'TensorFlow', 'spark':'Spark'})
jobs_df.head(10)

,job_title,company,location,job_description,Python,SQL,AWS,RESTFUL,Machine_Learning,Deep_Learning,Text_Mining,NLP,SAS,Tableau,Sagemaker,TensorFlow,Spark
c1e2abfc99160b71,Data Scientist (Junior),BDSA,"Remote in Louisville, CO","For example, combining public weather data wit...",True,False,False,False,False,False,False,False,True,False,False,False,False
67b11ca72ee88e80,Data Scientist III,GTA (Global Technology Associates),"Englewood, CO",Hands on experience developing video analytics...,True,False,True,False,True,False,False,False,False,False,False,False,False
1f343ce64c06dafb,Data Scientist,Visa,"Highlands Ranch, CO",Extract and understand data to form an opinion...,True,True,False,False,False,False,False,False,True,True,False,False,False
8697cd05eb89bfc8,Data Scientist,ISSAC Corp,"Colorado Springs, CO 80919 (Northwest Colorado...",Develop new machine learning and machine reaso...,True,False,False,False,True,True,False,False,False,False,False,False,False
e6d3a931c0a3ee80,Associate Data Scientist,GutCheck,"Remote in Denver, CO",Basic understanding in data wrangling and mini...,False,False,False,False,True,False,False,True,False,False,False,False,False
24ad9400b1d12fb0,Forecasting Data Scientist,DispatchHealth,"Remote in Denver, CO","Additionally, we are looking for someone with ...",True,True,False,False,False,False,False,False,False,False,False,True,True
75df6b591c581084,Data & Applied Scientist II,Microsoft,"Denver, CO 80237 (Hampden South area)",5+ yrs. experience in applied data science and...,True,True,True,False,True,True,False,True,False,False,False,False,False
0d2c7c663b98e9eb,Data Scientist,SHGT,"Denver, CO","Ability to analyze data to find insights, Visu...",True,True,True,False,False,False,False,True,False,True,True,False,True
a03d0907ec9d0acc,Data Scientist - Utility Reliability (Jr - Sr)...,E Source,"Hybrid remote in Denver, CO",E Source develops groundbreaking data science ...,True,False,False,False,False,False,False,False,True,False,False,False,False
9526e92027ac3b7b,Data Scientist,"City of Grand Junction, Colorado","Grand Junction, CO 81506",The successful candidate will be proficient wi...,False,False,False,False,True,False,False,False,False,False,False,False,False


# Q2(1 point) Save you DataFrame to a pickle file name *indeed_job_co.pkl*. 
   Load this pkl file in dataFrame and use this dataFrame for answering following questions.

   <font color='red'>upload the pickle file(indeed_job_co.pkl) along with solution notebook to the canvas</font>

In [8]:
## saving my df to pickle file
import pickle
jobs_df.to_pickle("./indeed_jobs_co.pkl")  

In [9]:
#reopening file 
with open('indeed_jobs_co.pkl', 'rb') as f:
    job_df = pickle.loads(f.read())
job_df.head()

,job_title,company,location,job_description,Python,SQL,AWS,RESTFUL,Machine_Learning,Deep_Learning,Text_Mining,NLP,SAS,Tableau,Sagemaker,TensorFlow,Spark
c1e2abfc99160b71,Data Scientist (Junior),BDSA,"Remote in Louisville, CO","For example, combining public weather data wit...",True,False,False,False,False,False,False,False,True,False,False,False,False
67b11ca72ee88e80,Data Scientist III,GTA (Global Technology Associates),"Englewood, CO",Hands on experience developing video analytics...,True,False,True,False,True,False,False,False,False,False,False,False,False
1f343ce64c06dafb,Data Scientist,Visa,"Highlands Ranch, CO",Extract and understand data to form an opinion...,True,True,False,False,False,False,False,False,True,True,False,False,False
8697cd05eb89bfc8,Data Scientist,ISSAC Corp,"Colorado Springs, CO 80919 (Northwest Colorado...",Develop new machine learning and machine reaso...,True,False,False,False,True,True,False,False,False,False,False,False,False
e6d3a931c0a3ee80,Associate Data Scientist,GutCheck,"Remote in Denver, CO",Basic understanding in data wrangling and mini...,False,False,False,False,True,False,False,True,False,False,False,False,False


<font size = "6" color='red'> Use pandas functionality to answer question 3</font>
# Q 3 a(1 point) Which city has maximum job posting.



In [10]:
# calculates the most frequent city in the location column
job_df['location'].mode()

0    Denver, CO
Name: location, dtype: object

In [11]:
#using value counts of pandas to show it is the most frequent
job_df['location'].value_counts()


Denver, CO                                           16
Remote in Denver, CO                                  8
Denver, CO 80202 (Union Station area)                 6
Boulder, CO                                           6
Hybrid remote in Denver, CO                           6
                                                     ..
Broomfield, CO 80021 (Interlocken area)               1
Denver, CO 80211 (Sunnyside area)                     1
Remote in Denver, CO 80202+1 location                 1
Temporarily Remote in Englewood, CO                   1
Denver, CO 80217 (Central Business District area)     1
Name: location, Length: 79, dtype: int64

# Q 3 b(1.5 point) - Top 3 most demanding skills(like Python, AWS, SQL ...)



In [12]:
value = []

cols = ['Python', 'SQL', 'AWS', 'RESTFUL', 'Machine_Learning', 'Deep_Learning', 'Text_Mining', 
        'NLP', 'SAS', 'Tableau', 'Sagemaker', 'TensorFlow', 'Spark']
for col in cols:
    #needed if column does not have a TRUE
    try: 
        value.append(job_df[col].value_counts()[1])
    except KeyError:
        value.append(0) 
        
Data= {'skill': cols, 'value': value}
df = pd.DataFrame(Data)

In [13]:
df = df.sort_values(by=['value'], ascending = False)
df.head(3)

,skill,value
0,Python,99
4,Machine_Learning,78
1,SQL,72


# Q3 c(.5 point) What other questions you would like to ask  based on indeed data?

This is a free response question.

I would like to see salary information and required years of experience from the job posting data. 